In [1]:
import time
import random

import gammapy
import logging
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", message="invalid value encountered in multiply")
warnings.filterwarnings("ignore", message="invalid value encountered in less")
warnings.filterwarnings("ignore", category=FutureWarning)
import click
import math
import os
import numpy as np
import astropy.units as u
import naima
from astropy.time import Time
from astropy.coordinates import Angle, SkyCoord, EarthLocation, AltAz
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from pathlib import Path
from regions import CircleSkyRegion
import itertools
import psutil
from scipy.stats import norm

from gammapy.irf import load_cta_irfs
from gammapy.maps import WcsGeom, MapAxis, WcsNDMap, Map
from gammapy.modeling.models import PowerLawSpectralModel, ExpCutoffPowerLawSpectralModel, PowerLawNormSpectralModel, NaimaSpectralModel, LogParabolaSpectralModel
from gammapy.modeling.models import PointSpatialModel, GaussianSpatialModel
from gammapy.modeling.models import SkyModel, BackgroundModel, FoVBackgroundModel 
from gammapy.modeling.models import Models, TemplateSpatialModel #SkyDiffuseCube
from gammapy.modeling import Fit
from gammapy.data import Observation
from gammapy.datasets import MapDataset
from gammapy.makers import MapDatasetMaker, SafeMaskMaker
from gammapy.estimators import ExcessMapEstimator
from gammapy.datasets import (
    Datasets,
    SpectrumDataset,
    SpectrumDatasetOnOff,
    FluxPointsDataset,
)
from gammapy.estimators import FluxPoints
from gammapy.estimators import FluxPointsEstimator

log = logging.getLogger(__name__)

In [ ]:
#!pip install h5py

!conda run -n gammapy conda install psutil

In [2]:
def prepare_cta_irfs(ra, dec):
    """Prepares CTA irfs"""
    log.info("        Preparing CTA IRFs")
    spatial_model_position_sim = SkyCoord(ra * u.deg, dec * u.deg, frame='icrs') # OBS POS!
    energy_axis = MapAxis.from_edges(
        np.logspace(-1, 2.2, 31), unit="TeV", name="energy", interp="log"
    )
    energy_axis_true = MapAxis.from_edges(
        np.logspace(-1.5, 2.3, 62), unit="TeV", name="energy_true", interp="log"
    )

    geom = WcsGeom.create(
        skydir=spatial_model_position_sim, binsz=0.02, width=(3, 3), frame="icrs", axes=[energy_axis]
    )

    # Create an in-memory observation
    POINTING = SkyCoord(ra * u.deg, dec * u.deg + offset, frame='icrs', unit='deg')
    obs = Observation.create(pointing=POINTING, livetime=LIVETIME, irfs=irfs)
    # Make the MapDataset
    empty = MapDataset.create(geom, energy_axis_true=energy_axis_true, name="dataset-simu")
    maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])
    maker_safe_mask = SafeMaskMaker(methods=["offset-max"], offset_max=4.0 * u.deg)
    dataset = maker.run(empty, obs)
    dataset = maker_safe_mask.run(dataset, obs)

    exposure = dataset.exposure
    background_model = dataset.background_model
    psf = dataset.psf
    edisp = dataset.edisp
    bkg_model = FoVBackgroundModel(dataset_name="dataset-simu")

    return dataset, exposure, bkg_model, psf, edisp 

In [3]:
def prepare_spatial_models(ra, dec, extension):
    """Defines simulated and fitted skymodels"""
        # Simulated models

    if(extension > 0.0):
        
        spatial_model_sim = GaussianSpatialModel(lon_0= ra * u.deg, lat_0= dec * u.deg, sigma= extension * u.deg)
       
        spatial_model_fit = GaussianSpatialModel(lon_0= ra * u.deg, lat_0= dec * u.deg, sigma= 0.2 * u.deg)
        spatial_model_fit.sigma.frozen = False
        spatial_model_fit.sigma.min = 0.05
        spatial_model_fit.sigma.max = spatial_model_fit.lat_0.value + 0.3

    else:
        
        spatial_model_sim = PointSpatialModel(lon_0= ra * u.deg, lat_0= dec * u.deg)
        spatial_model_fit = PointSpatialModel(lon_0= ra * u.deg, lat_0= dec * u.deg)

    spatial_model_fit.lon_0.frozen = False
    spatial_model_fit.lat_0.frozen = False

    spatial_model_fit.lon_0.min = spatial_model_fit.lon_0.value - 0.2
    spatial_model_fit.lon_0.max = spatial_model_fit.lon_0.value + 0.2
    spatial_model_fit.lat_0.min = spatial_model_fit.lat_0.value - 0.2
    spatial_model_fit.lat_0.max = spatial_model_fit.lat_0.value + 0.2

    return spatial_model_sim, spatial_model_fit

In [4]:
def create_gammaray_fit_models():
    """Returns gamma-ray spectral models"""

    # Powerlaw model                                                                                                                                                                                   
    spectral_model_pl_fit = PowerLawSpectralModel(index=2.0, amplitude="2.0e-12 cm-2 s-1 TeV-1", reference="1 TeV")
    spectral_model_pl_fit.amplitude.min = 1e-17
    spectral_model_pl_fit.amplitude.max = 1e-5
    spectral_model_pl_fit.reference.frozen = True
    spectral_model_pl_fit.index.min = 0.0
    spectral_model_pl_fit.index.max = 4.0

    # ECPL model                                                                                                                                                                                       
    spectral_model_ecpl_fit = ExpCutoffPowerLawSpectralModel(index=2.0, amplitude="2.0e-12 cm-2 s-1 TeV-1", reference="1 TeV", lambda_="0.01 TeV-1", alpha=1.0)
    spectral_model_ecpl_fit.amplitude.min = 1e-17
    spectral_model_ecpl_fit.amplitude.max = 1e-5
    spectral_model_ecpl_fit.index.min = 0.0
    spectral_model_ecpl_fit.index.max = 4.0
    spectral_model_ecpl_fit.reference.frozen = True
    spectral_model_ecpl_fit.alpha.frozen = True
    spectral_model_ecpl_fit.lambda_.frozen = False
        
    #LogParabola model
    spectral_model_lpb_fit = LogParabolaSpectralModel(alpha=2.3, amplitude="1e-12 cm-2 s-1 TeV-1", reference=1 * u.TeV, beta=0.5)
    spectral_model_lpb_fit.amplitude.min = 1e-17
    spectral_model_lpb_fit.amplitude.max = 1e-5
    spectral_model_lpb_fit.alpha.min = 0.0
    spectral_model_lpb_fit.alpha.max = 4.0
    spectral_model_lpb_fit.beta.min  = -5.0 # Not sure if its a reasonable choice! Check later!
    spectral_model_lpb_fit.beta.max  = 5.0
    spectral_model_lpb_fit.reference.frozen = True

    return spectral_model_pl_fit, spectral_model_ecpl_fit, spectral_model_lpb_fit


In [5]:
def create_particle_model(particle_type, spect_type, norm, index, cutoff, distance, ngas, seed_photon, ep_max, ee_max):
    
    if(particle_type == 'Had'):
        
        normalized_proton_spect = normalize_proton_model(spect_type, norm, index, cutoff, distance, ngas, ep_max)
        nh = ngas / u.cm**3
        
        if(spect_type == 'ECPL'):
            proton_distribution_sim = naima.models.ExponentialCutoffPowerLawH(
                amplitude=normalized_proton_spect.parameters["amplitudeH"].value / u.eV, 
                e_0=10 * u.TeV, 
                alpha=index, 
                lambda_=(1/cutoff) / u.TeV
            )

        else:
                proton_distribution_sim = naima.models.PowerLaw(
                amplitude=normalized_proton_spect.parameters["amplitude"].value / u.eV, 
                e_0=10 * u.TeV, 
                alpha=index, 
            )
            

        radiative_model_fit = naima.radiative.PionDecay(
            proton_distribution_sim,
            nh = nh,
            Epmax = ep_max,
        )
        
    else:
        
        normalized_electron_spect = normalize_electron_model(spect_type, norm, index, cutoff, distance, seed_photon, ee_max)

        seed_photon_fields = seed_photon 
        
        if(spect_type == 'ECPL'):

            electron_distribution_sim = naima.models.ExponentialCutoffPowerLawL(
                amplitude=normalized_electron_spect.parameters["amplitudeL"].value / u.eV, 
                e_0=1 * u.TeV, 
                alpha=index, 
                lambda_=(1/cutoff) / u.TeV
            )
            
        else:
            
                electron_distribution_sim = naima.models.PowerLaw(
                amplitude=normalized_electron_spect.parameters["amplitude"].value / u.eV, 
                e_0=1 * u.TeV, 
                alpha=index, 
            )

        radiative_model_fit = naima.radiative.InverseCompton(
            electron_distribution_sim,
            seed_photon_fields = seed_photon,
            Eemax = ee_max
        )
    
    distance = distance * u.kpc    
    particle_gamma_fit_model = NaimaSpectralModel(radiative_model_fit, distance=distance)
    
    if particle_type == 'Had':
        particle_gamma_fit_model.parameters["e_0H"].frozen = True
        particle_gamma_fit_model.parameters["alphaH"].min = 1.0
        particle_gamma_fit_model.parameters["alphaH"].max = 5.0
        particle_gamma_fit_model.parameters["amplitudeH"].min = 0
    
        if(spect_type == 'ECPL'):
            particle_gamma_fit_model.parameters["betaH"].frozen = True
            particle_gamma_fit_model.parameters["betaH"].frozen = True
            particle_gamma_fit_model.parameters["lambda_H"].min = -1
            #particle_gamma_fit_model.parameters["lambda_H"].min = 1e-5
            particle_gamma_fit_model.parameters["lambda_H"].max = 1.0
    
    else:
        particle_gamma_fit_model.parameters["e_0L"].frozen = True
        particle_gamma_fit_model.parameters["alphaL"].min = 1.0
        particle_gamma_fit_model.parameters["alphaL"].max = 5.0
        particle_gamma_fit_model.parameters["amplitudeL"].min = 0
    
        if(spect_type == 'ECPL'):
            particle_gamma_fit_model.parameters["betaL"].frozen = True
            particle_gamma_fit_model.parameters["betaL"].frozen = True
            particle_gamma_fit_model.parameters["lambda_L"].min = -1
            #particle_gamma_fit_model.parameters["lambda_L"].min = 1e-5
            particle_gamma_fit_model.parameters["lambda_L"].max = 1.0

    return particle_gamma_fit_model

In [6]:
def plot_spectrum(model, result, label, color):
    spec = model.spectral_model
    energy_range = [0.1, 160] * u.TeV
    spec.plot(
        energy_range=energy_range, energy_power=2, label=label, color=color
    )
    spec.plot_error(energy_range=energy_range, energy_power=2, color=color)

In [7]:
def plot_par_profiles(dataset, results):

    fit = Fit(dataset, store_trace=True)
    total_stat = results[0]
    for par in dataset.models.parameters:
        if par.frozen is False:
            profile = fit.stat_profile(parameter=par)
            plt.plot(
                profile[f"{par.name}_scan"], profile["stat_scan"] - total_stat
            )
            plt.xlabel(f"{par.unit}")
            plt.ylabel("Delta TS")
            plt.title(f"{par.name}: {par.value} +- {par.error}")
            plt.show()
            plt.close()

In [8]:
def normalize_proton_model(spect_type, norm, index, cutoff, distance, ngas, ep_max):
    """Normalize the model via the differential flux at 1 TeV (in Crab units), 
     instead of using the model amplitude, as usually done in Naima."""
    
    crab_unit = "3.84e-11 cm-2 s-1 TeV-1" # Crab flux at 1 TeV
    
    def model_to_be_normalized(x):
        
        if(spect_type == 'ECPL'):
            
            particle_distribution = naima.models.ExponentialCutoffPowerLawH(
                amplitude=x / u.eV, 
                e_0=10 * u.TeV, 
                alpha=index, 
                lambda_=(1/cutoff) / u.TeV
            )
            
        else:
            
            particle_distribution = naima.models.PowerLaw(
                amplitude=x / u.eV, 
                e_0=10 * u.TeV, 
                alpha=index, 
            )
            
        radiative_model = naima.radiative.PionDecay(
                particle_distribution,
                nh = ngas / u.cm**3,
                Epmax = 1e5 * u.PeV,
        )
        return NaimaSpectralModel(radiative_model=radiative_model, distance=distance*u.kpc)
        
    def f_pl(x):
        a = (model_to_be_normalized(x)(1*u.TeV) / crab_unit).to("") - norm
        return a
    #amplitude = fsolve(f_pl, 1e32)[0]
    amplitude = fsolve(f_pl, 1)[0]
    np.testing.assert_almost_equal(f_pl(amplitude), 0)
    return model_to_be_normalized(amplitude)

In [9]:
def normalize_electron_model(spect_type, norm, index, cutoff, distance, seed_photon, ee_max):
    
    crab_unit = "3.84e-11 cm-2 s-1 TeV-1" # Crab flux at 1 TeV
    
    def model_to_be_normalized(x):
        
        if(spect_type == 'ECPL'):
            
            particle_distribution = naima.models.ExponentialCutoffPowerLawL(
                amplitude=x / u.eV, 
                e_0=1 * u.TeV, 
                alpha=index, 
                lambda_=(1/cutoff) / u.TeV
            )
        else:
            
            particle_distribution = naima.models.PowerLaw(
                amplitude=x / u.eV, 
                e_0=1 * u.TeV, 
                alpha=index, 
            )            
            
        radiative_model = naima.radiative.InverseCompton(
                particle_distribution,
                seed_photon_fields = seed_photon,
                Eemax = 1000 * u.TeV,
        )
        return NaimaSpectralModel(radiative_model=radiative_model, distance=distance*u.kpc)
        
    def f_pl(x):
        a = (model_to_be_normalized(x)(1*u.TeV) / crab_unit).to("") - norm
        return a

    #amplitude = fsolve(f_pl, 1e32)[0]
    amplitude = fsolve(f_pl, 1)[0]
    print(f_pl(amplitude))
    np.testing.assert_almost_equal(f_pl(amplitude), 0)
    return model_to_be_normalized(amplitude)

In [10]:
def fit_dataset_3D(gammapy_event_file, model_fit):

        """ Fit a 3D dataset assuming a model """  
        maps = MapDataset.read(gammapy_event_file)
        dataset = maps.copy(name="dataset-simu")
   
        dataset.models = model_fit
        model_fit["dataset-simu-bkg"].spectral_model.norm.frozen = False
        model_fit["dataset-simu-bkg"].spectral_model.tilt.frozen = True
        model_fit["diffuse"].spectral_model.norm.frozen = False
        model_fit["diffuse"].spectral_model.tilt.frozen = True
        
        refit = True
        fit_counter = 0
        tolerance   = 0.1 

        while(refit):
            fit = Fit([dataset])
            result = fit.run(optimize_opts={"print_level": print_level, "tol": tolerance, "strategy": strategy})
            
            if(fit_counter == fit_counter_max):
                
                results_tuple = (
                result.total_stat, 
                False, 
                result.parameters,
                )
                print("Minimization problem after all trials !")
                return results_tuple, dataset
            
            if(result.success==True):
                print('Minimization successful')
                refit = False
            else:
                fit_counter = fit_counter + 1
                tolerance   = tolerance * 2
                print('Minimization failed --> Counter = ' + str(fit_counter) + ' increasing tolerance to '+str(tolerance))
                continue
        
        results_tuple = (
            result.total_stat, 
            result.success, 
            result.parameters,
        )

        log.info(result.parameters.to_table())

        return results_tuple, dataset

In [11]:
def simulate_dataset(model, livetime, dataset, background_model, diffuse, norm, index, cutoff):
    """Simulate n_obs instances of same the 3D dataset""" 
    
    modelsim = Models([model, background_model, diffuse])
        
    dataset.models = modelsim

    t = int( time.time() * 1000.0 )
    tt = ((t & 0xff000000) >> 24) + ((t & 0x00ff0000) >>  8) + ((t & 0x0000ff00) <<  8) + ((t & 0x000000ff) << 24)
    log.info(tt)

    dataset.fake(random_state=tt)
    out_gammapy_event_sim = prefix + 'gammapy_simulation_event_file_proton_norm-'+str(norm)+'_index-'+str(index)+'_cutoff-'+str(cutoff)+'.fits.gz'
    dataset.write(out_gammapy_event_sim, overwrite=True)    
    return dataset, out_gammapy_event_sim#, dataset

In [12]:
def get_spectral_flux_points(e_min, e_max, n_points, dataset, source_name, filename=None):
    
    # Create spectral bins log equal space between e_min and e_max for n spectral points
    # Source name is the name of the component in dataset (see how skymodels are defined, we put names there)
    energy_edges = np.logspace(np.log10(e_min), np.log10(e_max), num_points+1) * u.TeV
    fpe = FluxPointsEstimator(energy_edges=energy_edges, source=source_name)
    flux_points = fpe.run(datasets=dataset)
    flux_points.write(filename, overwrite=True)
    
    return flux_points    

In [13]:
def plot_flux_points_and_model(flux_points, model, ul_threshold, energy_power):
    
    # To plot flux points + model in different ways...
    plt.figure(figsize=(8, 6))
    flux_points.table["is_ul"] = flux_points.table["ts"] < ul_threshold
    ax = flux_points.plot(energy_power=energy_power, flux_unit="erg-1 cm-2 s-1", color="darkorange")
    flux_points.to_sed_type("e2dnde").plot_ts_profiles(ax=ax)

    plt.figure(figsize=(8, 6))

    flux_points_dataset = FluxPointsDataset(data=flux_points, models=model)
    flux_points_dataset.plot_fit()

# WE START THE ANALYSIS BELOW
# SET DESIRED SIMULATION & ANALYSIS OPTIONS HERE

In [14]:
gerrit_simple = False # If true: Fix fit parameters to make things easier in a first place
hadronic_true = True # If false: True model is leptonic

LIVETIME = 50 * u.hour

# Note that before they were 'SouthAz' IRFs, I changed them now to 'AverageAz' IRFs.

#irf_filename = ("fits/Prod5-South-20deg-AverageAz-14MSTs37SSTs.18000s-v0.1.fits.gz")   
irf_filename = ("/home/oguzhan/Python3Workspace/CTA_IRFs/fits/Prod5-South-20deg-AverageAz-14MSTs37SSTs.18000s-v0.1.fits.gz")         

irfs     = load_cta_irfs(irf_filename)

offset   = 0.7 * u.deg # Simulation offset (degrees)
distance = 4.0         # Source distance (kpc)
ngas     = 10.0        # Gas density for hadronic emission (cm^-3)
seed_photon = 'CMB'    # IC seed photon field for leptonic emission. We have 'CMB', 'NIR' and 'FIR' (near or far infrared)

# Minuit fit options
print_level = 0   
strategy    = 2 
fit_counter_max = 5

prefix = ''
diffuse_emission_file = prefix + 'IEM_base_v2.fits'
diffuse_model = Map.read(diffuse_emission_file)
print("DONE READ DIFFUSE...")

FileNotFoundError: [Errno 2] No such file or directory: '/home/oguzhan/Python3Workspace/CTA_IRFs/fits/Prod5-South-20deg-AverageAz-14MSTs37SSTs.18000s-v0.1.fits.gz'

In [ ]:
ep_max = 1e5 * u.PeV
ee_max = 510 * u.TeV #This is default value in naima

# HADRONIC SIM MODEL
if hadronic_true:
    label = 'True Hadronic Model'
    norm = 0.1 # Crab unit, so its 100 mCrab (phi_0 at 1 TeV)
    index = 2.0
    cutoff = 3000
    particle_sim_model = create_particle_model('Had','ECPL', norm, index, cutoff, distance, ngas, seed_photon, ep_max, ee_max) #HADRONIC

else:
# LEPTONIC SIM MODEL
    label = 'True Leptonic Model'
    norm = 0.1 # Crab unit, so its 100 mCrab (phi_0 at 1 TeV)
    index = 3.0
    cutoff = 500
    particle_sim_model = create_particle_model('Lep','ECPL', norm, index, cutoff, distance, ngas, seed_photon, ep_max, ee_max) #LEPTONIC

# Just to plot the gamma-ray model from proton spectrum we want to simulate
plt.figure(figsize=(8, 6))
energy_range = [0.1,160]*u.TeV
energy_power = 2 #(0 for dNdE, 2 for E^2dN/dE)
particle_sim_model.plot(energy_range=energy_range, energy_power=energy_power, label=label)
plt.legend()

# SET SOURCE MORPHOLOGY

In [ ]:
# RADEC for LHAASO J1825
ra  = 276.45
dec = -13.45
extension = 0.0 #(Point source)

sim_dataset, exposure, background_model, psf, edisp = prepare_cta_irfs(ra, dec)

spatial_model_sim, spatial_model_fit = prepare_spatial_models(ra, dec, extension)

# Morphology models for particle fit
spatial_model_fit_proton_ecpl   = spatial_model_fit.copy()
spatial_model_fit_electron_ecpl = spatial_model_fit.copy()
# Morphology models for gamma fit
spatial_model_fit_gamma_pl      = spatial_model_fit.copy()
spatial_model_fit_gamma_ecpl    = spatial_model_fit.copy()
spatial_model_fit_gamma_lpb     = spatial_model_fit.copy()

# SET SIMULATION & ANALYSIS SOURCE MODELS BELOW

In [ ]:
# ------------------------------------------------------
# SIMULATION MODEL TRUE (source + background + diffuse)
# ------------------------------------------------------

sky_model_sim = SkyModel(spatial_model=spatial_model_sim, spectral_model=particle_sim_model, name="model-simu")

diffuse_roi      = diffuse_model.interp_to_geom(sim_dataset.exposure.geom)
diffuse_template = TemplateSpatialModel(diffuse_roi, normalize=False)
diffuse          = SkyModel(PowerLawNormSpectralModel(), diffuse_template, name="diffuse")    

# --------------------------------------------------
# BELOW ARE THE GAMMA-RAY FIT MODELS FOR 3D ANALYSIS
# --------------------------------------------------

spectral_model_pl_fit, spectral_model_ecpl_fit, spectral_model_lpb_fit = create_gammaray_fit_models()
gamma_sky_model_pl   = SkyModel(spatial_model=spatial_model_fit_gamma_pl, spectral_model=spectral_model_pl_fit, name="model-fit-gamma-pl",)
gamma_sky_model_ecpl = SkyModel(spatial_model=spatial_model_fit_gamma_ecpl, spectral_model=spectral_model_ecpl_fit, name="model-fit-gamma-ecpl",)
gamma_sky_model_lpb  = SkyModel(spatial_model=spatial_model_fit_gamma_lpb, spectral_model=spectral_model_lpb_fit, name="model-fit-gamma-lpb",)

gamma_sky_model_pl_full_3d   =  Models([gamma_sky_model_pl, background_model, diffuse])
gamma_sky_model_ecpl_full_3d =  Models([gamma_sky_model_ecpl, background_model, diffuse])
gamma_sky_model_lpb_full_3d  =  Models([gamma_sky_model_lpb, background_model, diffuse])

# --------------------------------------------------
# BELOW ARE THE PARTICLE FIT MODELS FOR 3D ANALYSIS
# --------------------------------------------------

norm_init   = 0.05
index_init  = 2.2
cutoff_init = 1000

proton_model_ecpl_fit      = create_particle_model('Had', 'ECPL', norm_init, index_init, cutoff_init, distance, ngas, seed_photon, ep_max, ee_max)
#proton_model_ecpl_fit.parameters["lambda_3"].value  = 1 / 1000
#proton_model_ecpl_fit.parameters["lambda_3"].frozen = True
if gerrit_simple:
    proton_model_ecpl_fit.parameters["alphaH"].value  = index
    #proton_model_ecpl_fit.parameters["alphaH"].frozen = True
    #proton_model_ecpl_fit.parameters['lambda_H'].value = 1./cutoff
    #proton_model_ecpl_fit.parameters['lambda_H'].frozen = True
    proton_model_ecpl_fit.parameters["lambda_H"].min = 1 / 5000.
    proton_model_ecpl_fit.parameters["lambda_H"].max = 1 / 1.
    
# Electron model spectral fit models
norm_init_lep   = 0.05
index_init_lep  = 3.0
cutoff_init_lep = 100

lepton_model_ecpl_fit      = create_particle_model('Lep', 'ECPL', norm_init_lep, index_init_lep, cutoff_init_lep, distance, ngas, seed_photon, ep_max, ee_max)

if gerrit_simple:
    lepton_model_ecpl_fit.parameters["alphaL"].value  = index
    #lepton_model_ecpl_fit.parameters["alphaL"].frozen = True
    #lepton_model_ecpl_fit.parameters['lambda_L'].value = 1./cutoff_init_lep
    #lepton_model_ecpl_fit.parameters['lambda_L'].frozen = True
    lepton_model_ecpl_fit.parameters["lambda_L"].min = 1 / 5000.
    lepton_model_ecpl_fit.parameters["lambda_L"].max = 1 / 1.

proton_sky_model_ecpl      = SkyModel(spatial_model=spatial_model_fit_proton_ecpl, spectral_model=proton_model_ecpl_fit, name="model-fit-proton-ecpl",)
lepton_sky_model_ecpl      = SkyModel(spatial_model=spatial_model_fit_electron_ecpl, spectral_model=lepton_model_ecpl_fit, name="model-fit-lepton-ecpl",)

proton_sky_model_ecpl_full_3d      =  Models([proton_sky_model_ecpl, background_model, diffuse])
lepton_sky_model_ecpl_full_3d      =  Models([lepton_sky_model_ecpl, background_model, diffuse])

# -------------------------------------------------
# BELOW ARE THE PARTICLE FIT MODELS FOR 1D ANALYSIS
# -------------------------------------------------

spatial_model_fit_proton_ecpl        = spatial_model_fit.copy()
proton_model_1d = proton_model_ecpl_fit.copy()
lepton_model_1d = lepton_model_ecpl_fit.copy()


# WE SIMULATE DATASET FOR TRUE PARTICLE MODEL

In [ ]:
try:

    dataset_sim, out_gammapy_event_sim = simulate_dataset(sky_model_sim, LIVETIME, sim_dataset, background_model, diffuse, norm, index, cutoff)
            
except ValueError:
             
    print("We got value error...")

# WE GO FOR GAMMA RAY ANALYSIS OF THE SIMULATED DATA
# WE TEST BETWEEN SPECTRAL MODELS AND CHOOSE THE BEST ONE

In [ ]:
print("===== Gamma PL Fit to Data ====")
print("----------------------")
print(" Gamma PL 3D Fit ")
print("----------------------")
gamma_pl, dataset_gamma_pl = fit_dataset_3D(out_gammapy_event_sim, gamma_sky_model_pl_full_3d) 
print('Likelihood : ', gamma_pl[0])
print('Fit Status : ', gamma_pl[1])
print(gamma_pl[2].to_table())

print("===== Gamma ECPL Fit to Data ====")
print("----------------------")
print(" Gamma ECPL 3D Fit ")
print("----------------------")
gamma_ecpl, dataset_gamma_ecpl = fit_dataset_3D(out_gammapy_event_sim, gamma_sky_model_ecpl_full_3d) 
print('Likelihood : ', gamma_ecpl[0])
print('Fit Status : ', gamma_ecpl[1])
print(gamma_ecpl[2].to_table())

print("===== Gamma LPB Fit to Data ====")
print("----------------------")
print(" Gamma LPB 3D Fit ")
print("----------------------")
gamma_lpb, dataset_gamma_lpb = fit_dataset_3D(out_gammapy_event_sim, gamma_sky_model_lpb_full_3d) 
print('Likelihood : ', gamma_lpb[0])
print('Fit Status : ', gamma_lpb[1])
print(gamma_lpb[2].to_table())

In [ ]:
#SOME STATISTICS
print("--------------------------------------")
TS_ecpl_over_pl = gamma_pl[0] - gamma_ecpl[0] 
print('TS (PL-ECPL) = ', TS_ecpl_over_pl)
print('Sigma (PL-ECPL) = ', math.sqrt(TS_ecpl_over_pl))
print("--------------------------------------")
TS_lpb_over_pl = gamma_pl[0] - gamma_lpb[0] 
print('TS (PL-LPB) = ', TS_lpb_over_pl)
print('Sigma (PL-LPB) = ', math.sqrt(TS_lpb_over_pl))

#PLOT GAMMA MODELS
plt.figure(figsize=(8, 6))
plot_spectrum(gamma_sky_model_pl, gamma_pl, label="PowerLaw Fit", color="red")
plot_spectrum(gamma_sky_model_ecpl, gamma_ecpl, label="ECPL Fit", color="blue")
plot_spectrum(gamma_sky_model_lpb, gamma_lpb, label="LogParabola Fit", color="green")

energy_range = [0.1,160]*u.TeV
energy_power = 2 #(0 for dNdE, 2 for E^2dN/dE)
particle_sim_model.plot(energy_range=energy_range, energy_power=energy_power, label="Simulated" , color="black", ls='--', lw=2)
plt.legend()

# WE OBTAIN SPECTRAL FLUX POINTS USING THE BEST SPECTRAL MODEL
# FOR NOW WE JUST TAKE THE BEST MODEL BETWEEN PL AND ECPL
# SET BINNING PARAMETERS

In [ ]:
# ESTIMATE FLUX POINTS FROM THIS MODEL
# HOW DO 1D FIT RESULTS EFFECT FROM BINNING ? (CHECK)

e_min, e_max = 0.1, 160 # logspace
num_points = 10 # Number of flux points we want to have (we can ask for a lot since we have 50h observatios)

threshold_ts = 25.0

if(TS_ecpl_over_pl > threshold_ts):

    source_name = "model-fit-gamma-ecpl" # Thats what we defined in SkyModels for hadronic fit
    flux_dataset = dataset_gamma_ecpl
    file_out = 'test_flux_points_gamma_ecpl.fits'
    data_model = gamma_sky_model_ecpl
    
else:
    source_name = "model-fit-gamma-pl" # Thats what we defined in SkyModels for hadronic fit
    flux_dataset = dataset_gamma_pl
    file_out = 'test_flux_points_gamma_pl.fits'
    data_model = gamma_sky_model_pl

gamma_flux_points = get_spectral_flux_points(e_min, e_max, num_points, flux_dataset, source_name, file_out)

# Print out flux points
gamma_flux_points.table_formatted

#plot stuff if you want
ul_threshold = 4 # This is TS, so flux points < 2 sigma significance will be upper limits
energy_power = 2
plot_flux_points_and_model(gamma_flux_points, data_model, ul_threshold, energy_power)

In [ ]:
def fit_dataset_1D(model, flux_points):

    dataset_1d = FluxPointsDataset(model, flux_points)

    refit = True
    fit_counter = 0
    tolerance   = 0.1 
    fit_counter_max = 25
    strategy = 1

    while(refit):
        fit_1d = Fit([dataset_1d])
        result_1d = fit_1d.run(optimize_opts={"print_level": print_level, "tol": tolerance, "strategy": strategy})
        
        if(fit_counter == fit_counter_max):
                
            results_tuple = (
            result_1d.total_stat, 
            False, 
            result_1d.parameters,
            )
            print("Minimization problem after all trials !")
            return results_tuple
            
        if(result_1d.success==True):
            print('Minimization successful')
            refit = False
        else:
            fit_counter = fit_counter + 1
            tolerance   = tolerance * 2
            print('Minimization failed --> Counter = ' + str(fit_counter) + ' increasing tolerance to '+str(tolerance))
            continue
            
    results_tuple = (
        result_1d.total_stat, 
        result_1d.success, 
        result_1d.parameters,
    )

    return results_tuple

In [ ]:
# This is to test 1D particle models fit to flux points
# I just read from the written fits file 
# 1D fit can be quite problematic if we set strategy to 2 (deep fit) and use low tolerance
# Strategy 1 works fine in general... 

flux_points_read = FluxPoints.read(file_out)

proton_model_ecpl_1d      = create_particle_model('Had', 'ECPL', norm_init, index_init, cutoff_init, distance, ngas, seed_photon, ep_max, ee_max)
lepton_model_ecpl_1d      = create_particle_model('Lep', 'ECPL', norm_init_lep, index_init_lep, cutoff_init_lep, distance, ngas, seed_photon, ep_max, ee_max)

proton_model_1d   = SkyModel(spectral_model=proton_model_ecpl_1d)
lepton_model_1d    = SkyModel(spectral_model=lepton_model_ecpl_1d)

In [ ]:
#Proton 1d fit
print('==== Proton 1D fit results ====')
proton_1d_results = fit_dataset_1D(proton_model_1d, flux_points_read)
print('Likelihood : ', proton_1d_results[0])
print('Fit Status : ', proton_1d_results[1])
print(proton_1d_results[2].to_table())

#Lepton 1d fit
print('==== Lepton 1D fit results ====')
lepton_1d_results = fit_dataset_1D(lepton_model_1d, flux_points_read)
print('Likelihood : ', lepton_1d_results[0])
print('Fit Status : ', lepton_1d_results[1])
print(lepton_1d_results[2].to_table())


In [ ]:
# We create the composite model after fitting previous two
# so it can initialize better (hopefully)

#Hadron-initiated composite model components
sub_proton_ecpl_1d_h = proton_model_ecpl_1d.copy()
sub_lepton_ecpl_1d_h = lepton_model_ecpl_1d.copy()

#Lepton-initiated composite model components
sub_proton_ecpl_1d_l = proton_model_ecpl_1d.copy()
sub_lepton_ecpl_1d_l = lepton_model_ecpl_1d.copy()

# Reduce normalization of leptonic component in hadron initiated composite model
sub_lepton_ecpl_1d_h.parameters["amplitudeL"].value = sub_lepton_ecpl_1d_h.parameters["amplitudeL"].value/5

# Reduce normalization of hadronic component in lepton initiated composite model
sub_proton_ecpl_1d_l.parameters["amplitudeH"].value = sub_proton_ecpl_1d_l.parameters["amplitudeH"].value/5

#Hadron-initiated composite model
composite_model_h = sub_proton_ecpl_1d_h + sub_lepton_ecpl_1d_h
composite_model_1d_h = SkyModel(spectral_model=composite_model_h)

#Lepton-initiated composite model
composite_model_l = sub_proton_ecpl_1d_l + sub_lepton_ecpl_1d_l
composite_model_1d_l = SkyModel(spectral_model=composite_model_l)

#Composite 1d fit
print('==== Composite 1D fit hadron-initialized results ====')
composite_1d_h_results = fit_dataset_1D(composite_model_1d_h, flux_points_read)
print('Likelihood Had-Ini: ', composite_1d_h_results[0])
print('Fit Status Had-Ini: ', composite_1d_h_results[1])
print(composite_1d_h_results[2].to_table())

print('==== Composite 1D fit lepton-initialized results ====')
composite_1d_l_results = fit_dataset_1D(composite_model_1d_l, flux_points_read)
print('Likelihood Lep-Ini: ', composite_1d_l_results[0])
print('Fit Status Lep-Ini: ', composite_1d_l_results[1])
print(composite_1d_l_results[2].to_table())

In [ ]:
plt.figure(figsize=(12, 8))

#plot_spectrum(lepton_sky_model_ecpl, lepton_ecpl, label="Composite", color="tab:blue")
energy_range = [0.1,160]*u.TeV
energy_power = 2 #(0 for dNdE, 2 for E^2dN/dE)
particle_sim_model.plot(energy_range=energy_range, energy_power=energy_power, label="Simulated" , color="black", ls='--', lw=2)
gamma_flux_points.plot(energy_power=energy_power, color="darkorange")
#proton_model_ecpl_1d.plot(energy_range=energy_range, energy_power=energy_power, label="Only Hadronic 1D" , color="red", ls='-', lw=2)
#proton_model_ecpl_1d.plot_error(energy_range=energy_range, energy_power=2, color='red')
#lepton_model_ecpl_1d.plot(energy_range=energy_range, energy_power=energy_power, label="Only Leptonic 1D" , color="blue", ls='-', lw=2)
#lepton_model_ecpl_1d.plot_error(energy_range=energy_range, energy_power=2, color='blue')

composite_model_h.plot(energy_range=energy_range, energy_power=energy_power, label="Composite 1D-Had" , color="magenta", ls='-', lw=2)
#composite_model_h.plot_error(energy_range=energy_range, energy_power=2, color='magenta')
sub_proton_ecpl_1d_h.plot(energy_range=energy_range, energy_power=energy_power, label="Sub-Proton H" , color="red", ls='--', lw=2)
sub_lepton_ecpl_1d_h.plot(energy_range=energy_range, energy_power=energy_power, label="Sub-Lepton H" , color="blue", ls='--', lw=2)

composite_model_l.plot(energy_range=energy_range, energy_power=energy_power, label="Composite 1D-Lep" , color="green", ls='-', lw=2)
#composite_model_l.plot_error(energy_range=energy_range, energy_power=2, color='green')
sub_proton_ecpl_1d_l.plot(energy_range=energy_range, energy_power=energy_power, label="Sub-Proton L" , color="red", ls='-.', lw=2)
sub_lepton_ecpl_1d_l.plot(energy_range=energy_range, energy_power=energy_power, label="Sub-Lepton L" , color="blue", ls='-.', lw=2)


plt.legend()

In [ ]:
#Test 1:
dev_lepton = lepton_1d_results[0]#-2. * np.ln(proton_ecpl[0]) 
dev_composite = composite_1d_results[0]#-2. * np.ln(composite_ecpl[0])

TS_LR1 = np.max([0, dev_lepton - dev_composite])
print('TS1='+str(TS_LR1))
print('significance for the presence of a hadronic component: ' + str(np.sqrt(TS_LR1))) 

#Test 2:
dev_proton = proton_1d_results[0]#-2. * np.ln(proton_ecpl[0]) 
dev_composite = composite_1d_results[0]#-2. * np.ln(composite_ecpl[0])

TS_LR2 = np.max([0, dev_proton - dev_composite])
print('TS2='+str(TS_LR2))
print('significance for the presence of a leptonic component: ' + str(np.sqrt(TS_LR2))) 

# BELOW ARE 3D ANALYSIS

In [ ]:
# Proton model fit to simulated data

print("===== Proton Fit to Data ====")
print("----------------------")
print(" Proton ECPL Free Fit ")
print("----------------------")
proton_ecpl, dataset_proton_ecpl = fit_dataset_3D(out_gammapy_event_sim, proton_sky_model_ecpl_full) 
print('Likelihood Free : ', proton_ecpl[0])
print('Fit Status Free : ', proton_ecpl[1])
print(proton_ecpl[2].to_table())

In [ ]:
# LOOK HOW IF FITS, amplitudeH < 0.1
# IF THE FIT IS BAD, REFIT OR RESIMULATE (FOR NOW)
# ITS QUITE IMPROTANT TO GET A GOOD FIT FOR THE COMBINED MODEL
# MOST OF TIMES YOU CAN GET BETTER FIT, IF YOU REFIT

plt.figure(figsize=(8, 6))

#plot_spectrum(lepton_sky_model_ecpl, lepton_ecpl, label="Composite", color="tab:blue")
plot_spectrum(proton_sky_model_ecpl, proton_ecpl, label="Hadron", color="tab:blue")
energy_range = [0.1,160]*u.TeV
energy_power = 2 #(0 for dNdE, 2 for E^2dN/dE)
particle_sim_model.plot(energy_range=energy_range, energy_power=energy_power, label="Simulated" , color="black", ls='--', lw=2)
plt.legend()

In [ ]:
print("===== Electron Fit to Data ====")
print("----------------------")
print(" Electron ECPL Free Fit ")
print("----------------------")
lepton_ecpl, dataset_lepton_ecpl = fit_dataset_3D(out_gammapy_event_sim, lepton_sky_model_ecpl_full) 
print('Likelihood Free : ', lepton_ecpl[0])
print('Fit Status Free : ', lepton_ecpl[1])
print(lepton_ecpl[2].to_table())

In [ ]:
# LEPTON FIT TO PURE HADRON IS NOT SO GOOD IN GENERAL, ALTHOUGH FIT IS FINE 
# CHECK amplitudeL, its actually higher than amplitudeH
# (ACTUALLY LAMBDA PREFERS NEGATIVE IN GENERAL, IF YOU CONSTRAIN IT, THE FIT GETS WORST)

plt.figure(figsize=(8, 6))
plot_spectrum(lepton_sky_model_ecpl, lepton_ecpl, label="Lepton", color="tab:Green")
energy_range = [0.1,160]*u.TeV
energy_power = 2 #(0 for dNdE, 2 for E^2dN/dE)
particle_sim_model.plot(energy_range=energy_range, energy_power=energy_power, label="Simulated" , color="black", ls='--', lw=2)
plt.legend()

In [ ]:
plot_par_profiles(dataset_lepton_ecpl, lepton_ecpl)

In [ ]:
# NOW WE CREATE COMPOSITE MODEL FROM FITTED HADRON & LEPTON MODELS 
# INITIALIZATION OF THESE MODELS SOMEHOW MATTERS IN FIT
# Note that these models already carry fit information, one can also copy sure

#composite_spectral_model = proton_model_ecpl_fit + lepton_model_ecpl_fit
# We have to make copy of this fitted models, then combine (otherwise original fit results change)
sub_lepton_model_ecpl_fit = lepton_model_ecpl_fit.copy()
sub_proton_model_ecpl_fit = proton_model_ecpl_fit.copy()

sub_proton_sky_model_ecpl      = SkyModel(spatial_model=spatial_model_fit_proton_ecpl, spectral_model=sub_proton_model_ecpl_fit, name="sub-model-fit-proton-ecpl",)
sub_lepton_sky_model_ecpl      = SkyModel(spatial_model=spatial_model_fit_electron_ecpl, spectral_model=sub_lepton_model_ecpl_fit, name="sub-model-fit-lepton-ecpl",)

composite_spectral_model = sub_lepton_model_ecpl_fit + sub_proton_model_ecpl_fit

composite_sky_model_ecpl = SkyModel(spatial_model=spatial_model_fit_proton_ecpl, spectral_model=composite_spectral_model, name="model-fit-composite-ecpl",)
composite_model_full     =  Models([composite_sky_model_ecpl, background_model, diffuse])


In [ ]:
# IF IT PRINTS FALSE AFTER 5 TRIALS, RUN AGAIN THIS CELL (OR INCREASE TOLERANCE IN FIT_DATASET_3D). 
# IT WILL FIND AN OK FIT, HOPEFULLY.
print("===== Composite Fit to Data ====")
print("----------------------")
print(" Composite ECPL Free Fit ")
print("----------------------")
composite_ecpl, dataset_composite_ecpl = fit_dataset_3D(out_gammapy_event_sim, composite_model_full) 
print('Likelihood Free : ', composite_ecpl[0])
print('Fit Status Free : ', composite_ecpl[1])
print(composite_ecpl[2].to_table())

In [ ]:
# THIS COMPARES ALL TOGETHER WITH SUBCOMPONENTS

plt.figure(figsize=(12, 8))

energy_power = 2 #(0 for dNdE, 2 for E^2dN/dER
energy_range = [0.1,160]*u.TeV
particle_sim_model.plot(energy_range=energy_range, energy_power=energy_power, label="Simulated" , color="black", ls='--', lw=2)
plot_spectrum(sub_proton_sky_model_ecpl, composite_ecpl, label="Hadronic Component", color="tab:Blue")
plot_spectrum(sub_lepton_sky_model_ecpl, composite_ecpl, label="Leptonic Componen", color="tab:Green")
plot_spectrum(composite_sky_model_ecpl, composite_ecpl, label="Composite", color="tab:Red")

plt.legend()

In [ ]:
plot_par_profiles(dataset_composite_ecpl, composite_ecpl)

# AIC model selection

In [ ]:
AIC_proton = proton_ecpl[0]#-2. * np.log(proton_ecpl[0]) # modulo terms which depend on the number of parameters
AIC_lepton = lepton_ecpl[0]#-2. * np.log(lepton_ecpl[0]) # modulo terms which depend on the number of parameters

# If Delta_AIC > 0, the proton model gives better predictions for the data than the lepton model
Delta_AIC = AIC_lepton - AIC_proton # difference is the only relevant quantity, in this case independent from number of parameters
print('delta AIC: ' + str(Delta_AIC))

# Composite model selection

Test 1:

- Null hypothesis: Only leptonic
- Alternative hypothesis: There is a hadronic component

Test 2:

- Null hypothesis: Only hadronic
- Alternative hypothesis: There is a leptonic component


Note1: Delta_AIC is very close to TS_LR1 if the hadronic model is clearly preferred. This holds by construction, 
but only if the hadronic model is really clearly preferred.

Note2: This is a Likelihood ratio test. There are also other tests. One can disucss all this. Also regarding robustness, i.e. when the true models are neither our simple leptonic nor our simple hadronic models. One idea is to use an F-test again, like in my pevatron HESS GPS paper. One needs spectral points for this instead of likelihoods. But that's a topic for a detailed study, later.

In [ ]:
#Test 1:
dev_lepton = lepton_ecpl[0]#-2. * np.ln(proton_ecpl[0]) 
dev_composite = composite_ecpl[0]#-2. * np.ln(composite_ecpl[0])

TS_LR1 = np.max([0, dev_lepton - dev_composite])
print('TS1='+str(TS_LR1))
print('significance for the presence of a hadronic component: ' + str(np.sqrt(TS_LR1)))

In [ ]:
#Test 2:
dev_proton = proton_ecpl[0]#-2. * np.ln(proton_ecpl[0]) 
dev_composite = composite_ecpl[0]#-2. * np.ln(composite_ecpl[0])

TS_LR2 = np.max([0, dev_proton - dev_composite])
print('TS2='+str(TS_LR2))
print('significance for the presence of a leptonic component: ' + str(np.sqrt(TS_LR2)))

In [ ]:
# MAYBE ONE CAN JUDGE THE CONTRIBUTION BY CHECKING THE INTEGRAL FLUX ?

energy_min = 0.1   * u.TeV 
energy_max = 160.0 * u.TeV
integral_proton = proton_model_ecpl_fit.integral(energy_min=energy_min, energy_max=energy_max)

int_proton = integral_proton.value

integral_lepton = lepton_model_ecpl_fit.integral(energy_min=energy_min, energy_max=energy_max)

int_lepton = integral_lepton.value

total = int_proton + int_lepton

ratio_hadron = (int_proton/total)*100

print(ratio_hadron)